In [91]:
from __future__ import print_function, division
import scipy
from keras.models import load_model
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import keras
import pandas as pd
import helpers as H
from tqdm import tqdm

from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import cv2

In [2]:
#Root directory of the project
ROOT_DIR = os.path.abspath(".")
MODEL_PATH = os.path.join(ROOT_DIR, "models")
TEST_DATASET_PATH = os.path.join(ROOT_DIR, "dataset")
TEST_DATASET_PATH

'E:\\FastMRI\\MRI-reconstruction\\dataset'

In [3]:
TEST_LOW_DIR = "singlecoil_test_3D_images/"
TEST_IMAGES_DIR_PATH = os.path.join(TEST_DATASET_PATH, TEST_LOW_DIR)
TEST_IMAGES_DIR_PATH

'E:\\FastMRI\\MRI-reconstruction\\dataset\\singlecoil_test_3D_images/'

In [4]:
len(os.listdir(TEST_IMAGES_DIR_PATH))

108

In [5]:
test_images = [os.path.join(TEST_IMAGES_DIR_PATH, x) for x in os.listdir(TEST_IMAGES_DIR_PATH)]
len(test_images)

108

### Load the model

In [6]:
model = load_model(os.path.join(MODEL_PATH, 'g_AB.h5'), custom_objects={'InstanceNormalization':InstanceNormalization})
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 320, 320, 1)  0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 160, 160, 32) 544         input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_9 (LeakyReLU)       (None, 160, 160, 32) 0           conv2d_11[0][0]                  
__________________________________________________________________________________________________
instance_normalization_7 (Insta (None, 160, 160, 32) 2           leaky_re_lu_9[0][0]              
_____________________________________

C:\Users\Aiman\Anaconda3\envs\sticker\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [75]:
##img = H.normalize(img)
#flat = img.flatten()
#min(flat), max(flat)

In [79]:
def predict(img_3d):
    
    recon = []
    # Make it 4D for inference
    img_3d = np.moveaxis(img_3d, 0, 2) # whc
    slices = img_3d.shape[-1]

    # iterate over each slice and save
    for i in range(slices):
        img = img_3d[:,:,i]
        img = H.normalize(img) # Normalize image
        img =  img[np.newaxis,:,:,np.newaxis]
        img = model.predict(img)
        img = np.squeeze(img, axis=0)
        img = np.reshape(img, (320, 320))
        recon.append(img)
    
    recon = np.array(recon)
    return recon
        

In [80]:
fname = test_images[1]
a = np.load(fname)

In [81]:
out_3d = predict(a)
out_3d.shape

(33, 320, 320)

In [86]:
fname[-18:]

'file1000036_v2.npy'

In [88]:
RESULTS = "results"
H.create_directory("{}/{}".format(TEST_DATASET_PATH, RESULTS))

In [89]:
RESULTS_PATH = os.path.join(TEST_DATASET_PATH, RESULTS)
RESULTS_PATH

'E:\\FastMRI\\MRI-reconstruction\\dataset\\results'

In [ ]:
for path in tqdm(test_images):
    name = path[-18:]
    #print(name)
    vol = np.load(path)
    output = predict(vol)
    np.save("{}/{}".format(RESULTS_PATH, name), output)

 40%|████████████████████████████████▎                                                | 43/108 [00:49<01:19,  1.22s/it]